## Interactive Preprocessing Visualization Notebook

This notebook provides an interactive visualization of the pre-processing steps for a single recording.

Import the necessary libraries. If any of these libraries are not installed, you can install them using pip install grnsuite.

In [1]:
import grnsuite.preprocessing as preprocessing
import grnsuite.spike_detection as spikes
import grnsuite.utils as utils
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import glob
from matplotlib.widgets import Button, RadioButtons
from IPython.display import display, clear_output

import matplotlib
matplotlib.use("TkAgg")  # Ensures the correct interactive backend is used

Load the parameters from the parameters.yaml file and load the file paths to your data files.

In [2]:
# Load parameters
params = utils.load_parameters('parameters.yaml')

# Get list of files to process
if params['process_mode'] == 'all': 
    # Get all .txt files in the data directory
    files = glob.glob(os.path.join("data", "*.txt"))
    filenames = [os.path.basename(f) for f in files]
else:
    # Get specific files from parameters if they exist
    filenames = []
    if 'recordings' in params:
        filenames = [f"{name}.txt" for name in params['recordings'].keys()]
    else:
        raise ValueError("No recordings specified in parameters.yaml and process_mode is not 'all'")

print(f"Found {len(filenames)} files to process:")
for i, f in enumerate(filenames):
    print(f"  {i-1+1}. {f}")

Found 9 files to process:
  0. 20231103-M05-sucr-100-Gal-A2-01.txt
  1. 20231106-M01-sucr-100-Gal-A1-02.txt
  2. 20231103-M04-sucr-100-Gal-A2-02.txt
  3. 20231106-F02-sucr-100-Gal-A2-02.txt
  4. 20231103-M05-sucr-100-Gal-A1-02.txt
  5. 20231103-M05-sucr-100-Gal-A3-02.txt
  6. 20231106-M01-sucr-100-Gal-A2-02.txt
  7. 20231103-M04-sucr-100-Gal-A1-02.txt
  8. 20231106-F02-sucr-100-Gal-A3-02.txt


Select a file to process.

In [3]:
# Let user select a file to process - should be a number between 0 and the number of files.
file_index = int(input(f"Select a file to process (1-{len(filenames)}): ")) - 1
filename = filenames[file_index]
print(f"\nProcessing {filename}...")


Processing 20231106-F02-sucr-100-Gal-A2-02.txt...


Set up paths to the input and output files.

In [4]:
# Set up paths
filepath = os.path.join("data", filename)
sample_name = filename.replace('.txt', '')
output_dir = os.path.join("results", sample_name)
os.makedirs(output_dir, exist_ok=True)

Get the metadata from the file name - note, this relies on the parameters.yaml file that will dicate the order of the metadata in the file name.

In [5]:
# Get metadata from filename
metadata = utils.parse_filename_metadata(sample_name, params)
print(f"Metadata: {metadata}")

Metadata: {'date': '20231106', 'animal_id': 'F02', 'stimulus': 'sucr', 'concentration': '100', 'location': 'Gal', 'sensillum_id': 'A2', 'replicate': '02'}


Make sure the metadata is correct. If it isn't, you can edit the parameters.yaml file to change the order of the metadata in the file name.


## Step 1: Load the raw data

In [6]:
# === STEP 1: Load Raw Data ===
print("\n=== STEP 1: Loading Raw Data ===")
raw_data = preprocessing._load_ephys_data(filepath)

# %matplotlib inline

# Visualize raw data
plt.figure(figsize=(12, 6))
plt.plot(raw_data, 'k-', alpha=0.7)
plt.title("Step 1: Raw Electrophysiology Data")
plt.xlabel("Sample Index")
plt.ylabel("Voltage")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()



=== STEP 1: Loading Raw Data ===


invalid command name "4567691136process_stream_events"
    while executing
"4567691136process_stream_events"
    ("after" script)
2025-03-13 19:09:59.131 Python[24104:15798443] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-13 19:09:59.131 Python[24104:15798443] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Step 2: Select the contact artifact

In [7]:
# === STEP 2: Contact Artifact Selection ===
print("\n=== STEP 2: Contact Artifact Selection ===")
print("Use the slider to adjust the starting point of the recording.")
print("The contact artifact should be at the beginning of the red section.")

matplotlib.use("TkAgg")  # Use TkAgg for interactive selection
# The function now handles backend switching internally
selected_signal = preprocessing.interactive_contact_selection(raw_data)




=== STEP 2: Contact Artifact Selection ===
Use the slider to adjust the starting point of the recording.
The contact artifact should be at the beginning of the red section.


invalid command name "4542679616process_stream_events"
    while executing
"4542679616process_stream_events"
    ("after" script)


Selected segment from index 59325 to 149325


In [8]:
# Display the result
plt.figure(figsize=(12, 6))
plt.plot(selected_signal, 'r-')
plt.title("Step 2: Selected Signal After Contact Artifact Selection")
plt.xlabel("Sample Index")
plt.ylabel("Voltage")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


print(f"Selected signal length: {len(selected_signal)} samples")

invalid command name "4699334912process_stream_events"
    while executing
"4699334912process_stream_events"
    ("after" script)


Selected signal length: 90000 samples


## Step 3: Filter the signal

In [9]:
# === STEP 3: Filtering ===
print("\n=== STEP 3: Filtering the Signal ===")
print(f"Applying bandpass filter ({params['filter_low']}-{params['filter_high']} Hz) and noise removal")

filtered_signal = preprocessing.filter_signal(selected_signal, params)

# Visualize filtering effects
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

# Original signal
axes[0].plot(selected_signal, 'k-', alpha=0.7)
axes[0].set_title("Before Filtering (Raw Selected Signal)")
axes[0].set_ylabel("Voltage")
axes[0].grid(True, alpha=0.3)

# After bandpass filter
bandpass_only = preprocessing._apply_filter(
    selected_signal, 
    [params['filter_low'], params['filter_high']], 
    'bandpass', 
    fs=params['sampling_rate']
)
axes[1].plot(bandpass_only, 'b-', alpha=0.7)
axes[1].set_title(f"After Bandpass Filter ({params['filter_low']}-{params['filter_high']} Hz)")
axes[1].set_ylabel("Voltage")
axes[1].grid(True, alpha=0.3)

# After noise removal
axes[2].plot(filtered_signal, 'g-', alpha=0.7)
axes[2].set_title("After Noise Removal (Final Filtered Signal)")
axes[2].set_xlabel("Sample Index")
axes[2].set_ylabel("Voltage")
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Filtered signal properties:")
print(f"  - Mean: {np.mean(filtered_signal):.4f}")
print(f"  - Std Dev: {np.std(filtered_signal):.4f}")


=== STEP 3: Filtering the Signal ===
Applying bandpass filter (100-1000 Hz) and noise removal


invalid command name "4542810176process_stream_events"
    while executing
"4542810176process_stream_events"
    ("after" script)


Filtered signal properties:
  - Mean: -0.0346
  - Std Dev: 53.9675


## Step 4: Zoom to region of interest

In [10]:
# === STEP 4: Zoom to Region of Interest ===
print("\n=== STEP 4: Zooming to Region of Interest ===")
print(f"Selecting portion from {params['offset_time']}s to {params['offset_time'] + params['analysis_length']}s")

# Zoom and automatically save the figure
zoomed_signal, current_time = preprocessing.zoom_to_region(
    filtered_signal, 
    params,
    output_dir=output_dir
)

# Visualize zooming effect
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Full filtered signal with highlighted region
time_full = np.arange(len(filtered_signal)) / params['sampling_rate']
axes[0].plot(time_full, filtered_signal, 'b-', alpha=0.5)

# Calculate indices for the zoomed region
start_idx = int(params['offset_time'] * params['sampling_rate'])
end_idx = int((params['offset_time'] + params['analysis_length']) * params['sampling_rate'])

# Highlight the zoomed region
axes[0].axvspan(params['offset_time'], params['offset_time'] + params['analysis_length'], 
                color='yellow', alpha=0.3)
axes[0].set_title("Full Filtered Signal with Highlighted Analysis Region")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Voltage")
axes[0].grid(True, alpha=0.3)

# Zoomed region
axes[1].plot(current_time, zoomed_signal, 'g-')
axes[1].set_title("Zoomed Region for Analysis")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Voltage")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Zoomed signal length: {len(zoomed_signal)} samples")
print(f"Time range: {current_time[0]:.2f}s to {current_time[-1]:.2f}s")


=== STEP 4: Zooming to Region of Interest ===
Selecting portion from 0.1s to 2.1s
Zoomed data figure saved to: results/20231106-F02-sucr-100-Gal-A2-02/zoomed_data.png


invalid command name "4706096768process_stream_events"
    while executing
"4706096768process_stream_events"
    ("after" script)


Zoomed signal length: 60000 samples
Time range: 0.10s to 2.10s


## Step 5: Signal Normalization

In [11]:
# === STEP 5: Signal Normalization ===
print("\n=== STEP 5: Signal Normalization (MAD) ===")
print("Normalizing signal using Median Absolute Deviation")

# Calculate median and MAD for display
median = np.median(zoomed_signal)
mad = np.median(np.abs(zoomed_signal - median))
mad_factor = 1.4826 * mad

mad_signal = preprocessing.normalize_signal(zoomed_signal)

# Visualize normalization effect
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Before normalization
axes[0].plot(current_time, zoomed_signal, 'b-')
axes[0].axhline(y=median, color='r', linestyle='-', label=f"Median = {median:.4f}")
axes[0].axhline(y=median + mad_factor, color='g', linestyle='--', 
                label=f"Median + MAD = {median + mad_factor:.4f}")
axes[0].axhline(y=median - mad_factor, color='g', linestyle='--', 
                label=f"Median - MAD = {median - mad_factor:.4f}")
axes[0].set_title("Before MAD Normalization")
axes[0].set_ylabel("Voltage")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# After normalization
axes[1].plot(current_time, mad_signal, 'g-')
axes[1].axhline(y=0, color='r', linestyle='-', label="Median = 0")
axes[1].axhline(y=1, color='g', linestyle='--', label="Median + MAD = 1")
axes[1].axhline(y=-1, color='g', linestyle='--', label="Median - MAD = -1")
axes[1].set_title("After MAD Normalization")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Normalized Voltage")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("MAD normalization statistics:")
print(f"  - Original median: {median:.4f}")
print(f"  - Original MAD: {mad_factor:.4f}")
print(f"  - Normalized median: {np.median(mad_signal):.4f}")
print(f"  - Normalized MAD: {np.median(np.abs(mad_signal - np.median(mad_signal))) * 1.4826:.4f}")

# Save the processed data
processed_data_path = os.path.join(output_dir, 'interactive_processed.csv')
preprocessing.save_results(mad_signal, current_time, processed_data_path)
print(f"\nSaved processed data to: {processed_data_path}")


=== STEP 5: Signal Normalization (MAD) ===
Normalizing signal using Median Absolute Deviation


invalid command name "4513513408process_stream_events"
    while executing
"4513513408process_stream_events"
    ("after" script)


MAD normalization statistics:
  - Original median: -5.5219
  - Original MAD: 27.7805
  - Normalized median: -0.1988
  - Normalized MAD: 1.0000
Successfully saved processed data to: results/20231106-F02-sucr-100-Gal-A2-02/interactive_processed.csv

Saved processed data to: results/20231106-F02-sucr-100-Gal-A2-02/interactive_processed.csv


## Step 6: Spike Detection

In [12]:
# === STEP 6: Interactive Spike Detection ===
print("\n=== STEP 6: Interactive Spike Detection ===")
print("Adjust the threshold slider to set the optimal spike detection threshold")

# Perform interactive threshold adjustment
manual_threshold = spikes.adjust_threshold(mad_signal)

# Detect spikes with the manual threshold
spikes_file, spike_times, spike_values = spikes.detect_spikes_manual_threshold(
    processed_data_path,
    output_dir,
    threshold=manual_threshold
)

# Plot the final result with detected spikes
plt.figure(figsize=(12, 6))
plt.plot(current_time, mad_signal, 'b-', alpha=0.7, label="Normalized Signal")
plt.scatter(spike_times, spike_values, color='r', s=50, marker='x', label="Detected Spikes")
plt.axhline(y=manual_threshold, color='g', linestyle='--', label=f"Threshold = {manual_threshold:.2f}")
plt.title("Final Result: Normalized Signal with Detected Spikes")
plt.xlabel("Time (s)")
plt.ylabel("Normalized Voltage")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Detected {len(spike_times)} spikes using threshold = {manual_threshold:.2f}")
print(f"Spike details saved to: {spikes_file}")


=== STEP 6: Interactive Spike Detection ===
Adjust the threshold slider to set the optimal spike detection threshold
Initial threshold calculated: 2.458646894704656
Data range: min=-5.276186642637373, max=11.380631244134365


invalid command name "4392232832process_stream_events"
    while executing
"4392232832process_stream_events"
    ("after" script)


Selected threshold: 2.458646894704656
Final threshold value: 2.458646894704656
Using thresholds: upper=2.4586, lower=1.9669
Detected 177 spikes using threshold = 2.46
Spike details saved to: results/20231106-F02-sucr-100-Gal-A2-02/detected_spikes.csv


## Step 7: Extract waveforms

In [13]:
# === STEP 7: Waveform Extraction and Analysis ===
print("\n=== STEP 7: Waveform Extraction and Analysis ===")
print("Extracting waveforms around each detected spike")

# Define waveform extraction parameters
pre_peak_ms = 1.5  # milliseconds before spike
post_peak_ms = 1.5  # milliseconds after spike

# Use the existing extract_waveforms function
waveforms_file = spikes.extract_waveforms(
    data_file=processed_data_path,
    spikes_file=spikes_file,
    output_dir=output_dir,
    pre_peak_length=pre_peak_ms,
    post_peak_length=post_peak_ms
)

# Load the saved waveforms
waveforms_df = pd.read_csv(waveforms_file)
waveforms = waveforms_df.values

# Use the enhanced plot_waveforms function that now includes average and std
print("Plotting waveforms with average and standard deviation:")
avg_waveform, std_waveform = spikes.plot_waveforms(
    waveforms, 
    pre_peak_ms=pre_peak_ms, 
    post_peak_ms=post_peak_ms,
    show_time_axis=True,
    output_dir=output_dir
)

# Create a raster plot to visualize spike timing
plt.figure(figsize=(12, 4))
for i, spike_time in enumerate(spike_times):
    plt.plot([spike_time, spike_time], [i-0.4, i+0.4], 'k-')
    
plt.title("Spike Raster Plot")
plt.xlabel("Time (s)")
plt.ylabel("Spike Number")
plt.xlim(current_time[0], current_time[-1])
plt.tight_layout()
plt.show()

# Calculate time axis for metric calculations
time_ms = np.linspace(-pre_peak_ms, post_peak_ms, waveforms.shape[1])

# Calculate waveform metrics
peak_idx = np.argmax(avg_waveform)
trough_idx = np.argmin(avg_waveform)
peak_amplitude = avg_waveform[peak_idx]
trough_amplitude = avg_waveform[trough_idx]
peak_to_trough = peak_amplitude - trough_amplitude

# Calculate timing metrics
peak_time = time_ms[peak_idx]
trough_time = time_ms[trough_idx]
peak_to_trough_duration = abs(trough_time - peak_time)

print(f"\nExtracted {waveforms.shape[0]} waveforms")
print(f"Waveform duration: {pre_peak_ms + post_peak_ms:.1f} ms ({waveforms.shape[1]} samples)")
print(f"Saved waveforms to: {waveforms_file}")

print("\nWaveform metrics:")
print(f"Peak amplitude: {peak_amplitude:.3f}")
print(f"Trough amplitude: {trough_amplitude:.3f}")
print(f"Peak-to-trough amplitude: {peak_to_trough:.3f}")
print(f"Peak time: {peak_time:.2f} ms")
print(f"Trough time: {trough_time:.2f} ms")
print(f"Peak-to-trough duration: {peak_to_trough_duration:.2f} ms")


=== STEP 7: Waveform Extraction and Analysis ===
Extracting waveforms around each detected spike
Plotting waveforms with average and standard deviation:
Waveform figure saved to: results/20231106-F02-sucr-100-Gal-A2-02/waveforms_plot.png


invalid command name "4726312064process_stream_events"
    while executing
"4726312064process_stream_events"
    ("after" script)



Extracted 177 waveforms
Waveform duration: 3.0 ms (90 samples)
Saved waveforms to: results/20231106-F02-sucr-100-Gal-A2-02/waveforms.csv

Waveform metrics:
Peak amplitude: 6.641
Trough amplitude: -2.523
Peak-to-trough amplitude: 9.165
Peak time: 0.02 ms
Trough time: 1.43 ms
Peak-to-trough duration: 1.42 ms
